# Textcuboid


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

## Read the data into a dataframe
import json
import pandas as pd
import numpy as np
import re
import string
import nltk


data=[]
with open('C:/Users/user/Desktop/bilm-tf-master/textdataset/News_Category_Dataset_v3.json', 'r') as f:
    for line in f:
        content = json.loads(line)
        data.append(content)
df = pd.DataFrame(data=data)

In [2]:
df = df[~df['short_description'].apply(lambda x: len(x)==0)]
print(df.shape)

#중복되는 카테고리 통합
df["category"] = df.category.replace(
    {
        "THE WORLDPOST": "WORLD NEWS",
        "WORLDPOST": "WORLD NEWS",
        "ARTS": "ARTS & CULTURE",
        "CULTURE & ARTS": "ARTS & CULTURE",
        "HEALTHY LIVING": "WELLNESS",
        "QUEER VOICES": "VOICES",
        "LATINO VOICES": "VOICES",
        "BLACK VOICES": "VOICES",
        "STYLE": "STYLE & BEATUY",
        "GREEN": "ENVIRONMENT",
        "TASTE": "FOOD & DRINK",
        "MONEY": "BUSINESS",
        "PARENTING": "PARENTS"
    }
)

df = df[['headline', 'short_description','category']]
df['input_data']= df.apply(lambda x: str(x['headline']) + str(x['short_description']), axis=1)

def clean_text(text):
    text=str(text).lower() #Converts text to lowercase
    text=re.sub('\d+', '', text) #removes numbers
    text=re.sub('\[.*?\]', '', text) #removes HTML tags
    text=re.sub('https?://\S+|www\.\S+', '', text) #removes url
    text=re.sub(r"["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", "", text) #removes emojis
    text=re.sub('[%s]' % re.escape(string.punctuation),'',text) #removes punctuations
    #text = re.sub('\n', '', text)
    #text = re.sub('\w*\d\w*', '', text)
    return text

df['clean_text']=df['input_data'].apply(clean_text)

class0=list(df[df['category'] == 'POLITICS'].sample(1000, random_state=1)['clean_text'])
class1=list(df[df['category'] == 'WELLNESS'].sample(1000, random_state=1)['clean_text'])
class2=list(df[df['category'] == 'ENTERTAINMENT'].sample(1000, random_state=1)['clean_text'])
class3=list(df[df['category'] == 'PARENTS'].sample(1000, random_state=1)['clean_text'])
class4=list(df[df['category'] == 'VOICES'].sample(1000, random_state=1)['clean_text'])
class5=list(df[df['category'] == 'STYLE & BEAUTY'].sample(1000, random_state=1)['clean_text'])
class6=list(df[df['category'] == 'TRAVEL'].sample(1000, random_state=1)['clean_text'])
class7=list(df[df['category'] == 'FOOD & DRINK'].sample(1000, random_state=1)['clean_text'])
class8=list(df[df['category'] == 'WORLD NEWS'].sample(1000, random_state=1)['clean_text'])
class9=list(df[df['category'] == 'BUSINESS'].sample(1000, random_state=1)['clean_text'])

X_list=class0+class1+class2+class3+class4+class5+class6+class7+class8+class9

y=[]
for i in range(10):
    y+=[i]*1000

clr_x_data=[]
pattern = '[^a-z ]'
for sen in X_list:
    clr_x_data.append(re.sub(pattern, ' ', sen))

X_list=[]
for sen in clr_x_data:
    X_list.append(' '.join(sen.split()))

train_idx=[]
for i in range(10):
    train_idx+=[j+i*1000 for j in range(800)]

test_idx=[]
for i in range(10):
    test_idx+=[j+800+i*1000 for j in range(200)]

x_train=[]
y_train=[]
for i in train_idx:
    x_train.append(X_list[i])
    y_train.append(y[i])

x_test=[]
y_test=[]
for i in test_idx:
    x_test.append(X_list[i])
    y_test.append(y[i])

to_txt_filter=x_train+x_test
y=list(y_train)+list(y_test)

(189815, 6)


## 2) textcuboid 분류

In [3]:
textcuboid=np.load('./1-Channel textcuboid_nc(bert).npy')

In [4]:
textcuboid_test=np.load('./1-Channel textcuboid_test_nc(bert).npy')

In [5]:
tmp = [[x,y] for x, y in zip(textcuboid, y_train)]
import random
random.shuffle(tmp)
textcuboid = [n[0] for n in tmp]
y_train = [n[1] for n in tmp]
textcuboid=np.array(textcuboid)
y_train=np.array(y_train)

In [6]:
import keras
from tensorflow.keras import layers

In [7]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
#     keras.callbacks.EarlyStopping(monitor="val_loss", patience=8, verbose=1),
]

In [8]:
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model


input_layer = Input(shape=(68, 768))
conv1 = Conv1D(1024, 1, padding='valid', activation='relu')(input_layer)
pooling = GlobalMaxPooling1D()(conv1)

x = Dense(1024, activation='relu')(pooling)
x = Dropout(0.5)(x)
output_layer = Dense(10, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output_layer)

In [9]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 68, 768)]         0         
                                                                 
 conv1d (Conv1D)             (None, 68, 1024)          787456    
                                                                 
 global_max_pooling1d (Globa  (None, 1024)             0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 1024)              1049600   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 10)                10250     
                                                             

In [11]:
x_train=textcuboid[1000:]
x_val=textcuboid[:1000]
y_train1=y_train[1000:]
y_val=y_train[:1000]

In [12]:
history = model.fit(x_train, y_train1,callbacks=callbacks, epochs=20,batch_size=256,validation_data=(x_val, y_val))

Epoch 1/20
28/28 [==============================] - 2s 27ms/step - loss: 4.4962 - accuracy: 0.3410 - val_loss: 1.1900 - val_accuracy: 0.6290 - lr: 0.0010
Epoch 2/20
28/28 [==============================] - 1s 19ms/step - loss: 1.0087 - accuracy: 0.6746 - val_loss: 0.8319 - val_accuracy: 0.7360 - lr: 0.0010
Epoch 3/20
28/28 [==============================] - 1s 19ms/step - loss: 0.7369 - accuracy: 0.7664 - val_loss: 0.7755 - val_accuracy: 0.7530 - lr: 0.0010
Epoch 4/20
28/28 [==============================] - 1s 19ms/step - loss: 0.5616 - accuracy: 0.8217 - val_loss: 0.7533 - val_accuracy: 0.7560 - lr: 0.0010
Epoch 5/20
28/28 [==============================] - 1s 18ms/step - loss: 0.4159 - accuracy: 0.8671 - val_loss: 0.7625 - val_accuracy: 0.7560 - lr: 0.0010
Epoch 6/20
28/28 [==============================] - 1s 19ms/step - loss: 0.2942 - accuracy: 0.9120 - val_loss: 0.7543 - val_accuracy: 0.7750 - lr: 0.0010
Epoch 7/20
28/28 [==============================] - 0s 18ms/step - loss: 0.1

In [13]:
y_test=np.array(y_test)

In [14]:
model.evaluate(textcuboid_test,y_test)

63/63 [==============================] - 0s 3ms/step - loss: 0.9222 - accuracy: 0.7545


[0.9222186207771301, 0.7544999718666077]

In [15]:
from keras.models import load_model
model = load_model('best_model.h5')
model.evaluate(textcuboid_test,y_test)

63/63 [==============================] - 0s 3ms/step - loss: 0.7906 - accuracy: 0.7460


[0.7906113862991333, 0.7459999918937683]

In [35]:
np.average([75.5, 74.8, 76.2])

75.5

In [34]:
#NC로 학습한 BERT
np.average([68.8, 68.8, 68.8])

68.8